In [ ]:
import pandas as pd
import numpy as np
import json
import re
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [ ]:
# 시각화 설정 (한글 깨짐 방지)
plt.rcParams['font.family'] = 'Malgun Gothic' # Windows 사용자
# plt.rcParams['font.family'] = 'AppleGothic' # Mac 사용자
plt.rcParams['axes.unicode_minus'] = False

In [ ]:
# 크롤링된 파일명 입력
# 예: reviews_1562700_200_{app_id}.json
filename_input = input("분석할 JSON 파일명을 입력하세요: ")

if not filename_input.lower().endswith('.json'):
    filename = filename_input + '.json'
else:
    filename = filename_input

base_path = "../dataSet"
full_path = os.path.join(base_path, filename)
absolute_path = os.path.abspath(full_path)
print(f"로드할 파일 경로: {full_path}")

try:
    with open(absolute_path, 'r', encoding='utf-8') as f:
        data = json.load(f)
    
    # JSON 구조에 따라 데이터 로드 방식 변경 (리뷰 리스트가 'reviews' 키 아래에 있다고 가정)
    reviews_data = data.get('reviews', data) if isinstance(data, dict) else data
    df = pd.DataFrame(reviews_data)
    
    # 게임 이름 추출 (파일명을 기준으로)
    game_name = filename.split('_')[-1].replace('.json', '')
    
    print(f"✅ '{game_name}' 데이터 로드 완료. 총 리뷰 수: {len(df)}개")
    print("\n--- 데이터 미리보기 ---")
    print(df.head())

except FileNotFoundError:
    print(f"❌ 오류: 파일을 찾을 수 없습니다: {full_path}. 경로를 확인해 주세요.")
    df = pd.DataFrame()
except Exception as e:
    print(f"❌ 데이터 로드 중 오류 발생: {e}")
    df = pd.DataFrame()

In [ ]:
# 성향별 키워드 사전 정의
KEYWORDS = {
    "narrative": ["스토리", "서사", "감동", "엔딩", "캐릭터", "몰입", "대사", "연출", "배경"],
    "freedom": ["자유도", "오픈월드", "탐험", "상호작용", "선택", "커스터마이징", "비선형"],
    "stability": ["최적화", "버그", "프레임", "렉", "튕김", "서버", "운영", "잔렉", "불안정"],
    "challenge": ["난이도", "컨트롤", "보스", "피지컬", "패턴", "도전", "소울", "어려움", "노력"]
}

def calculate_persona_vector(review_text):
    """단일 리뷰 텍스트를 분석하여 성향 벡터 (4축 점수)를 반환합니다."""
    scores = {k: 0 for k in KEYWORDS.keys()}
    total_hits = 0
    
    if pd.isna(review_text):
        return {k: 0.0 for k in KEYWORDS.keys()}

    # 텍스트 내 키워드 카운팅
    for category, words in KEYWORDS.items():
        for word in words:
            # 정규 표현식을 사용하여 해당 단어의 출현 횟수 카운트
            count = len(re.findall(word, review_text, re.IGNORECASE))
            scores[category] += count
            total_hits += count
            
    # 정규화 (전체 합이 1이 되도록 비중 계산)
    if total_hits == 0:
        return {k: 0.0 for k in KEYWORDS.keys()}
    
    persona_vector = {k: round(v / total_hits, 3) for k, v in scores.items()}
    return persona_vector

In [ ]:
# 각 리뷰 텍스트에 함수 적용하여 'persona_vector' 컬럼 생성
if not df.empty:
    df['persona_vector'] = df['review_text'].apply(calculate_persona_vector)

    # 벡터 딕셔너리를 개별 컬럼으로 분리
    vector_df = df['persona_vector'].apply(pd.Series)
    vector_df.columns = ['S_' + col for col in vector_df.columns]
    df = pd.concat([df, vector_df], axis=1)

    print("\n✅ 성향 벡터 계산 완료.")
    print(df[['review_text', 'S_narrative', 'S_freedom', 'S_stability', 'S_challenge']].head(4))
else:
    print("분석할 DataFrame이 비어 있어 성향 벡터 계산을 건너뜁니다.")

In [ ]:
# 성향 컬럼들만 선택하여 평균 계산
if not df.empty:
    persona_columns = ['S_narrative', 'S_freedom', 'S_stability', 'S_challenge']
    average_persona = df[persona_columns].mean().sort_values(ascending=False)
    
    plt.figure(figsize=(8, 5))

    sns.barplot(
        x=average_persona.index, 
        y=average_persona.values, 
        hue=average_persona.index, # x 변수를 hue에 할당
        palette="viridis",
        legend=False # 범례(legend)는 필요 없으므로 False로 설정
    )

    plt.title(f'{game_name} 리뷰어들의 평균 성향 벡터', fontsize=15)
    plt.ylabel('평균 가중치', fontsize=12)
    plt.xticks(rotation=0)
    plt.show()

In [ ]:
# 플레이 시간을 시간 단위로 변환
if not df.empty:
    df['playtime_hours'] = df['playtime_forever'] / 60
    
    # 플레이 시간에 따른 분위기 파악
    plt.figure(figsize=(10, 6))
    sns.boxplot(x='voted_up', y='playtime_hours', data=df, showfliers=False) # 이상치 제외
    plt.title(f'{game_name} 추천/비추천별 플레이 시간 분포', fontsize=15)
    plt.xlabel('추천 여부 (False: 비추천, True: 추천)', fontsize=12)
    plt.ylabel('플레이 시간 (시간)', fontsize=12)
    plt.show()

In [ ]:
if not df.empty:
    # 분석 결과 파일 경로 생성
    output_csv_filename = f"analyzed_{game_name}_reviews.csv"
    output_full_path = os.path.join(base_path, output_csv_filename)
    absolute_full_path = os.path.abspath(output_full_path)
    df.to_csv(absolute_full_path, index=False, encoding='utf-8')
    print(f"\n📂 분석 결과가 '{absolute_full_path}' 파일에 저장되었습니다.")